**Install and import dependency**

In [ ]:
%pip install opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as pyplot
import time
import mediapipe as mp

**check webcam and keypoint of mediapipe holistic**

In [3]:
mp_holistic = mp.solutions.holistic # holistic module
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [4]:
def mediapipe_detection(image,model):
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) # color conversion
  image.flags.writeable = False
  results = model.process(image) # process image
  image.flags.writeable = True
  image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR) # color conversion
  return image,results

In [5]:
def draw_landmarks(image,result):
  mp_drawing.draw_landmarks(image,result.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
  mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)


In [6]:
def draw_styles_landmarked(image,result):
  mp_drawing.draw_landmarks(image,result.face_landmarks,mp_holistic.FACEMESH_CONTOURS,
                            mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                            mp_drawing.DrawingSpec(color=(80,256,10),thickness=1,circle_radius=1))
  mp_drawing.draw_landmarks(image,result.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
  mp_drawing.draw_landmarks(image,result.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)


In [7]:
cap = cv2.VideoCapture(0) #default camera
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holitic:
  while cap.isOpened():
    ret,frame = cap.read() #read frame from webcam
    
    #make pipe detection
    image,result = mediapipe_detection(frame, holitic)
    draw_styles_landmarked(image,result)

    cv2.imshow("Camera",image) #show on screen
    if cv2.waitKey(10) & 0xff == ord('q'): # if q or close is pressed break
      break

  cap.release()  # disconnect webcam
  cv2.destroyAllWindows() #close window

**Extract Value**

In [8]:
def extract_keypoint(result):
  pose = np.array([[res.x,res.y,res.z,res.visibility] for res in result.pose_landmarks.landmark]).flatten() if result.pose_landmarks else np.zeros(33*4)
  face = np.array([[res.x,res.y,res.z] for res in result.face_landmarks.landmark]).flatten() if result.face_landmarks else np.zeros(468*3)
  left_hand = np.array([[res.x,res.y,res.z] for res in result.left_hand_landmarks.landmark]).flatten() if result.left_hand_landmarks else np.zeros(21*3)
  right_hand = np.array([[res.x,res.y,res.z] for res in result.right_hand_landmarks.landmark]).flatten() if result.right_hand_landmarks else np.zeros(21*3)
  return np.concatenate([pose,face,left_hand,right_hand])

**Folder and default variable Setup**

In [9]:
DATA_PATH = os.path.join("MP_DATA")
with open("word_list.txt") as file:
  action = file.readlines()
for i,word in enumerate(action):
  action[i] = word.replace("\n","")
actions = np.array(action) # meaning of the pose
no_sequences = 30 # take 30 video to collect data for each action
sequence_length = 30 # 30 frames to detect what is going on

In [ ]:
for action in actions:
  for sequence in range(no_sequences):
    try:
      os.makedirs(os.path.join(DATA_PATH,action,str(sequence)))
    except:
      pass

**Data Collection**

In [ ]:
#video feed
cap = cv2.VideoCapture(0) #default camera
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holitic:
  for action in actions:
    for sequence in range(no_sequences):
      for frame_num in range(sequence_length):
        ret,frame = cap.read() #read frame from webcam
        #make pipe detection
        
        image,result = mediapipe_detection(frame, holitic)
        draw_landmarks(image,result)

        #show which pic is being collected
        if frame_num == 0:
          cv2.putText(image,"3",(120,200),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),4,cv2.LINE_AA)
          cv2.waitKey(1000)
          cv2.putText(image,"2",(120,200),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),4,cv2.LINE_AA)
          cv2.waitKey(1000)
          cv2.putText(image,"1",(120,200),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),4,cv2.LINE_AA)
          cv2.waitKey(1000)
        cv2.putText(image,f"{action} : {sequence}",(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_4)

        cv2.imshow("Camera",image) #show on screen

        # write data to path
        keypoint = extract_keypoint(result)
        npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
        np.save(npy_path,keypoint)

        if cv2.waitKey(10) & 0xff == ord('q'): # if q or close is pressed break
          break

  cap.release()  # disconnect webcam
  cv2.destroyAllWindows() #close window

In [ ]:
#in case the camera cant start
cap.release()  # disconnect webcam
cv2.destroyAllWindows() #close window

In [ ]:
#all available action
actions

In [ ]:
#video input
file_name = "D:/test.mp4" #path of this file
action = "hello"
if action in actions:
  #make new file
  sequence = 0
  while os.path.exists(os.path.join(DATA_PATH,action,str(sequence))):
    sequence+=1

  cap = cv2.VideoCapture(file_name)
  if cap.isOpened():
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holitic:
      for frame_num in range(sequence_length):
        ret,frame = cap.read() #read frame from webcam

        #make pipe detection
        image,result = mediapipe_detection(frame, holitic)
        draw_landmarks(image,result)
        #show which pic is being collected
      
        cv2.putText(image,f"{action}",(15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_4)

        cv2.imshow("Camera",image) #show on screen

        # write data to path
        keypoint = extract_keypoint(result)
        npy_path = os.path.join(DATA_PATH,action,str(sequence),str(frame_num))
        np.save(npy_path,keypoint)

        if cv2.waitKey(10) & 0xff == ord('q'): # if q or close is pressed break
          break

      cap.release()  # disconnect webcam
      cv2.destroyAllWindows() #close window
  else:
    print("Video Error")
else:
  print("action invalid")



**Data preprocessing, Create Label and Features**

In [13]:
#import dependencies
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {lable:num for num,lable in enumerate(np.array(os.listdir(os.path.join(DATA_PATH))))}

In [14]:
sequences,lables = [], []
for action in os.listdir(os.path.join(DATA_PATH)):
  action_file = os.listdir(os.path.join(DATA_PATH,action))
  for sequence in range(len(action_file)): #should change to the number of file
    window = []
    for frame_num in range(sequence_length):
      res = np.load(os.path.join(DATA_PATH,action,str(sequence),f"{frame_num}.npy"))
      window.append(res)
    sequences.append(window)
    lables.append(label_map[action])
    
X = np.array(sequences)
y = to_categorical(lables).astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05)

KeyboardInterrupt: 

In [ ]:
np.array(sequences).shape

In [ ]:
# X_train.shape
y_train

**Training the AI**

In [12]:
# import dependencies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
actions = np.array(os.listdir(os.path.join(DATA_PATH)))
actions.shape[0]

In [ ]:
model = Sequential() #
model.add(LSTM(64,return_sequences=True,activation='relu',input_shape=(30,1662))) #frame and all data shape
model.add(LSTM(128,return_sequences=True,activation='relu')) 
model.add(LSTM(64,return_sequences=False,activation='relu')) # return false because next layer is dense
model.add(Dense(64,activation='relu')) 
model.add(Dense(32,activation='relu')) 
model.add(Dense(actions.shape[0],activation='softmax')) # return output by how many action do we have => [0.7,0.1,0.2] / likely be 'hello'
#loss = categorical => multiple class 
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])
#epochs increase relatively the number of data 
model.fit(X_train,y_train,epochs=1000,callbacks=tb_callback)

In [ ]:
#watch the log
%cd logs/train
%tensorboard --logdir=.

In [ ]:
model.summary()

**prediction**

In [ ]:
res = model.predict(X_test)
res
# actions[np.argmax(res[4])] 
# actions[np.argmax(y_test[4])] 
# these two should be the same

**save weight**

In [ ]:
model.save("action.h5")

In [11]:
# reload model weight
model.load_weights('action.h5')

NameError: name 'model' is not defined

**Evaluatiion using Confusion Matrix and Accuracy**

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()
multilabel_confusion_matrix(ytrue,yhat)
#     [[True_Positive,True_Negative]
#     [False_Positive,False_Negative]]
#   True_Negative,False_Positive should be really low

In [ ]:
accuracy_score(ytrue,yhat)

**Covert file to tfjs**

In [ ]:
%tensorflowjs_converter --input_format=keras action.h5 tfjs

**Testing**

In [ ]:
from scipy import stats

In [ ]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()